In [0]:
from keras.models import Sequential, Model
from keras.models import load_model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import imgaug as ia
from tqdm import tqdm
from imgaug import augmenters as iaa
import numpy as np
import pickle
import os, cv2
import pandas as pd

Using TensorFlow backend.


Check the dimensions of image

In [0]:
image = cv2.imread("H:/datasets/Grid/level3/training_images/1.png",0)
image.shape

(480, 640)

Read the images as gray scale image.
Make a list X_data of np arrays of all the training images and simultaneously keep checking if all the images have the same dimension or not
if all training images have same dimension then k=0 else k!=0

In [0]:
X_data = []
k=0
for i in range(1,24001):
    myFile = "H:/datasets/Grid/level3/training_images/" + str(i) + ".png"
    if i%500==0:
        print(myFile)
    image = cv2.imread(myFile,0)
    if image.shape != (480,640):
      k=k+1
    X_data.append (image)
    
print(k)

We found one image of dimension (640, 480) so we rotated that image by 90 degree  and replaced the original image with this rotated image into the training image dataset

Now convert this list into an np array and save this array into the memory 

In [0]:
b = X_data

In [0]:
del X_data

In [0]:
b = np.array(b)

In [0]:
np.save("H:/datasets/Grid/level3/gray.npy",b)


In [0]:
b = np.load('H:/datasets/Grid/level3/gray.npy')

In [0]:
b.dtype

dtype('float32')

In [0]:
b = b.astype(np.float32)
b.dtype

dtype('float32')

In [0]:
b = b.reshape(24000,480,640,1)

Make a data generator and apply feature normalization i.e. mean=0 and std deviation=1

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)
datagen.fit(b)

Load the training bounding box csv file

In [0]:
boundary = pd.read_csv("H:/datasets/Grid/level3/training_set.csv")
boundary.head(5)

,image_name,x1,x2,y1,y2
0,JPEG_20160517_140621_1000651031832.png,78,446,83,422
1,1473231510890DeeplearnS11456.png,73,567,146,351
2,1476881557883Untitled1.png,13,561,124,362
3,1474264827890cobalt-beyond-the-sea-maxi-dress3...,0,640,140,330
4,JPEG_20160711_113321_1000235741992.png,217,442,65,465


In [0]:
df = boundary.copy()
del df["image_name"]

In [0]:
y = df.values

In [0]:
y = y.astype(np.float32)
y.dtype

dtype('float32')

Create the model

In [0]:
    model = Sequential()

    model.add(Conv2D(24, (5, 5), padding='same', input_shape=b.shape[1:], name='block1_conv1'))
    model.add(BatchNormalization()) 
    model.add(Activation('elu'))


    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

    model.add(Conv2D(36, (5, 5), padding='same', name='block2_conv2'))
    model.add(BatchNormalization()) 
    model.add(Activation('elu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

    model.add(Conv2D(48, (5, 5), padding='same', name='block3_conv1'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))

    model.add(Conv2D(64, (3, 3), padding='same', name='block3_conv3'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))


    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

    model.add(Conv2D(64, (3, 3), padding='same', name='block4_conv1'))
    model.add(BatchNormalization()) 
    model.add(Activation('elu'))



    model.add(Conv2D(128, (3, 3), padding='same', name='block4_conv4'))
    model.add(BatchNormalization()) 
    model.add(Activation('elu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))
    
    model.add(Conv2D(128, (3, 3), padding='same', name='block5_conv4'))
    model.add(BatchNormalization()) 
    model.add(Activation('elu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))
    
    model.add(Conv2D(256, (3, 3), padding='same', name='block6_conv4'))
    model.add(BatchNormalization()) 
    model.add(Activation('elu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block6_pool'))
    
    model.add(Conv2D(256, (3, 3), padding='same', name='block7_conv4'))
    model.add(BatchNormalization()) 
    model.add(Activation('elu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block7_pool'))

   

    model.add(Flatten())

    model.add(Dense(500))
    model.add(BatchNormalization()) 
    model.add(Activation('elu'))
    
    model.add(Dense(100))
    model.add(BatchNormalization()) 
    model.add(Activation('elu'))
   
    model.add(Dense(4))


In [0]:
model.summary()

Compile the model  using mean_absolute_error as the loss function and Adam optimizer with learning rate = 1

In [0]:
model.compile(loss='mean_absolute_error', optimizer=Adam(lr=1))

Use fit_generator to train the model

In [0]:
model.fit_generator(datagen.flow(b, y, batch_size=50),
                    steps_per_epoch=len(b) / 50, epochs=20)

Continue to train the model with learning rate 0.1 and 0.01 for better accuracy

In [0]:
model.compile(loss='mean_absolute_error', optimizer=Adam(lr=0.1))

In [0]:
model.fit_generator(datagen.flow(b, y, batch_size=50),
                    steps_per_epoch=len(b) / 50, epochs=15)

In [0]:
model.compile(loss='mean_absolute_error', optimizer=Adam(lr=0.01))

In [0]:
model.fit_generator(datagen.flow(b, y, batch_size=50),
                    steps_per_epoch=len(b) / 50, epochs=15)

Save the weights into the memory

In [0]:
model.save("H:/datasets/Grid/level3/gridweights.h5")